<h1> Main.ipynb </h1>
<h2> 1. Import libraries and modules </h2>

In [1]:
import numpy as np
import pandas as pd

from src.Data_Retriever import DataRetriever as DatRet

from src.Optimization.Markowitz_PT import MarkowitzPT as MPT

from src.Optimization.Environment import PortfolioEnvironment as PorEnv
from src.Optimization.RLModelCompilation import RL_Model as RLM
# from src.Optimization.NeuralNet import CustomNeuralNet as CusNN
# from src.Optimization.NeuralNet import CustomSACPolicy as CSACP
from src.Optimization.NeuralNet import SectorLSTMAttentionPolicy
from src.Result.Menchero_OGA import MencheroOGA as MOGA
from src.Result.IndPortResults import GenerateResult as GR
from src.Result.OverviewResults import ResultConveyor as RC

<h2> 2. Define operating variables </h2>

In [2]:
# Define necessary non-fixed variables
trading_n = 400
history_usage = 252
n_sectors = 6
n_stocks_per_sector = 4

# For RL algorithm
history_usage_RL = 30
rolling_reward_window = 10

<h2> 3. Define stock dataframe </h2>

In [3]:
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", "SUBC.OL", "BWO.OL",]
ticker_df["Seafood (food)"] = ["ORK.OL", "MOWI.OL", "SALM.OL", "LSG.OL"]
ticker_df["Materials"] = ["NHY.OL", "YAR.OL", "RECSI.OL", "BRG.OL"]
ticker_df["Technologies"] = ["TEL.OL", "NOD.OL", "ATEA.OL", "BOUV.OL"]
ticker_df["Financial"] = ["STB.OL", "DNB.OL", "GJF.OL", "AKER.OL"]
ticker_df["Shipping"] = ["WAWI.OL", "SNI.OL", "BELCO.OL", "ODF.OL"]
ticker_df

,Petroleum,Seafood (food),Materials,Technologies,Financial,Shipping
0,EQNR.OL,ORK.OL,NHY.OL,TEL.OL,STB.OL,WAWI.OL
1,AKRBP.OL,MOWI.OL,YAR.OL,NOD.OL,DNB.OL,SNI.OL
2,SUBC.OL,SALM.OL,RECSI.OL,ATEA.OL,GJF.OL,BELCO.OL
3,BWO.OL,LSG.OL,BRG.OL,BOUV.OL,AKER.OL,ODF.OL


<h2> 4. Define ESG array </h2>

In [4]:
# Defining ESG scores for respective securities
esg_scores = [36.6, 35.3, 17.9, 18, 
              18, 21.2, 18.7, 29.2, 
              15.7, 25.6, 25.6, 18.4, 
              19.8, 13.8, 18.1, 19, 
              17.2, 14, 17.2, 19.5, 
              19.7, 21.2, 26.8, 19.3]

<h2> 5.-7. (Non-necessary for reproducability) </h2>
<h3>Data retrieval</h3>
<h3>Benchmark optimization (MPT)</h3>
<h3>RL optimization x4</h3>
<br>
All of the data has been generated an stored in csvs. Therefore, I can comment out the code lines below.

In [5]:
# # Retrieve data from yf API: y-m-d
# data = DatRet(ticker_df, "2013-01-01", "2024-01-01")
# # In function below, set log=True to check for data availability
# data.retrieve_data()

In [6]:

# # Generate benchmark weights thorugh MPT using Sharpe ratio
# benchmark = MPT(history_usage, trading_n)
# # IMPORTANT: In order to see  the effect of the weights, algo exclude last observation from optimization
# benchmark.frequency_optimizing()

In [9]:
# 

In [7]:
# objectives = ["Return", "Sharpe", "Sortino", "Sterling", "Return", "Sharpe", "Sortino", "Sterling"]
# esg_compliancy = [True, True, True, True, False, False, False, False]
objectives = ["Sortino"]
esg_compliancy = [True]

for i in range(len(objectives)):
    reinforcement = RLM(esg_scores, 
                        objective=objectives[i],
                        history_usage=history_usage_RL,
                        rolling_reward_window=rolling_reward_window,
                        total_timesteps=1000,
                        esg_compliancy=esg_compliancy[i],
                        )
    reinforcement.train_model()
    reinforcement.test_model()

96
Combined data columns: 96
Combined data columns: 96
Using cpu device


c:\Users\Jakob\OneDrive\Dokumenter\Masteroppgave\MyVenv\Lib\site-packages\stable_baselines3\common\buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 1.73GB > 1.02GB
  warnings.warn(


Observation shape: (3600,)


TypeError: cannot unpack non-iterable NoneType object

<h2> 8. Attribution analysis </h2>

In [ ]:
paths = ["Return_esg_True", "Sharpe_esg_True",
         "Sortino_esg_True","Sterling_esg_True",
         "Return_esg_False", "Sharpe_esg_False",
         "Sortino_esg_False","Sterling_esg_False",]

analysis_list = []
for i in range(len(paths)):
    att_anal = GR(paths[i],
            n_sectors, n_stocks_per_sector,
            trading_n,
            esg_scores, 
            ticker_df.columns)
    att_anal.friple_frequency_analysis()
    analysis_list.append(att_anal)

----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----


<h2> X. Ad hoc analysis </h2>

In [ ]:
theta = RC(analysis_list, trading_n)
theta.convey_results()